In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error

plt.rcParams["figure.figsize"] = (18, 9)

In [ ]:
train_path = "andan-2022-autumn-lab-4/train.csv"
test_path = "andan-2022-autumn-lab-4/test_x.csv"

In [ ]:
train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)

In [ ]:
from math import log2

plt.scatter(train_data[train_data.osm_train_stop_closest_dist < 200]["osm_train_stop_closest_dist"].apply(log2), train_data[train_data.osm_train_stop_closest_dist < 200].per_square_meter_price )
plt.xlabel('osm_train_stop_closest_dist')
plt.ylabel('per_square_meter_price')

plt.show()



In [ ]:

population_mean = 2607030.2667519217
towns_with_metro = {
    "Москва", "Санкт-Петербург", "Новосибирск", "Екатеринбург", "Казань", "Нижний Новгород", "Самара"
}
# train_data[train_data["city"] == "Самара"]["osm_subway_closest_dist"].mean() # везде не больше 4

search = [ "per_square_meter_price" ]
params = [
    "city",

    "total_square",
    "osm_train_stop_closest_dist",
    "osm_subway_closest_dist",
    "osm_city_nearest_population", # have NaN of 35 count
    "osm_amenity_points_in_0.01",
    "osm_building_points_in_0.01",
    "osm_catering_points_in_0.01",
    "osm_crossing_points_in_0.01",
    "osm_culture_points_in_0.01",
    "osm_finance_points_in_0.01",
    "osm_healthcare_points_in_0.01",
    "osm_historic_points_in_0.01",
    "osm_hotels_points_in_0.01",
    "osm_leisure_points_in_0.01",
    "osm_offices_points_in_0.01",
    "osm_shops_points_in_0.01",
    "osm_train_stop_points_in_0.01",
    "osm_transport_stop_points_in_0.01",
    "reform_count_of_houses_1000",
]
# new_params = [
#     "is_msk",
#     "is_spb"
# ]

_dict = dict()
_train_data = pd.read_csv(train_path)
_test_data = pd.read_csv(test_path)
for name in params:
    if name in [ "city" ]: # ignore
        continue
    _dict[name] = (min(min(_train_data[name]), min(_test_data[name])), max(max(_train_data[name]), max(_test_data[name])))


# FIXME переделать под заранее определенные min и max
# def minmax(col, _min, _max):
def minmax(col, name):

    _min, _max = _dict[name]

    col[col < _min] = 0
    col[_max < col] = 1
    # from sklearn.preprocessing import minmax_scale
    # col[(_min < col) & (col < _max)] = minmax_scale(col[(_min < col) & (col < _max)], copy=False)
    col[(_min < col) & (col < _max)] = (col[(_min < col) & (col < _max)] - _min) / (_max - _min)
    return col

def city_handler(s):
    return [
        1 if s in towns_with_metro else 0, # metro in city
        1 if s == "Москва" else 0,
        1 if s == "Санкт-Петербург" else 0,
    ]

def line_handler(line):
    ma = []
    have_metro = False
    for name in params:
        if name in [
                    # 'per_square_meter_price',
                    ]: # ignore
            continue
        elif name in ['city']:
            ma += city_handler(line[name])
            if line[name] in towns_with_metro:
                have_metro = True
        elif name in ["osm_subway_closest_dist"]:
            ma += [line[name]] if have_metro else [0]
            # ma += [minmax(line[name], name)] if have_metro else [0]
        else:
            ma += [line[name]]
            # ma += [minmax(line[name], name)]
    return ma

def getRegression(data, is_train=True):

    data = data[params + (search if is_train else [])]

    # data = data[data.osm_city_nearest_population.notna()]
    data.osm_city_nearest_population.fillna(population_mean, inplace=True)

    # data = data.assign(
        ### rewrite
        # total_square=minmax(data["total_square"], 0),
        # osm_train_stop_closest_dist=minmax(data["osm_train_stop_closest_dist"], 0, 200),
        # osm_subway_closest_dist=minmax(data["osm_subway_closest_dist"], 0, 2000),
        ### new
        # is_msk=np.where((data['city'] == "Москва"), 1, 0),
        # is_spb=np.where((data['city'] == "Санкт-Петербург"), 1, 0),
        # is_city_mil_nearby=np.where((data['osm_city_nearest_population'] > 1e6), 1, 0),
        # osm_city_closest_dist=minmax(data["osm_city_closest_dist"]),
    # )

    # data = data[params + new_params]

    X = []
    for i in range(len(data)):
        line = data.iloc[i]
        X += [line_handler(line)]
        # print(X)

    # print(X[0])
    X = np.array(X)
    # print(X[0])
    return X, data['per_square_meter_price'] if is_train else None

In [ ]:
reg = LinearRegression(copy_X=False, n_jobs=-1)

In [ ]:
train_data_original = pd.read_csv(train_path)
train_data = train_data_original.copy()

In [ ]:
X, per_square_meter_price = getRegression(train_data)

In [ ]:
X_with_org_original = list(zip(X, list(per_square_meter_price)))

In [ ]:
errors = []
def pred(X, per_square_meter_price, test_x, test_y, reg):
    reg.fit(X, per_square_meter_price)
    prc = reg.predict(test_x)
    # errors.append(mean_absolute_percentage_error(test_y, prc))
    errors.append(2 ** mean_absolute_percentage_error(2 ** test_y, 2 ** prc))
    print(errors[-1])

for i in range(10):
    import random
    X_with_org = X_with_org_original.copy()
    random.shuffle(X_with_org)
    train_comb = X_with_org[int(len(X_with_org)*0.8):]
    test_comb = X_with_org[:int(len(X_with_org)*0.8)]
    X, per_square_meter_price = zip(*train_comb)
    test_x, test_y = zip(*test_comb)

    pred(X, per_square_meter_price, test_x, test_y, reg)

print("Mean:", sum(errors)/len(errors))
# 1) 3.08928935118498
# 2) add start/end to dists
# 2.990469893742038
# 3) remove city
# 3.9515472764739865

# 5) without minmax
# 2.535033998987495

# 6) with more params
# 2.291704818166524

### Отсюда пойдут предсказания

In [ ]:
train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)

In [ ]:
X, per_square_meter_price = getRegression(train_data)

In [ ]:
reg.fit(X, per_square_meter_price)

In [ ]:
X_test, _ = getRegression(test_data, False)

In [ ]:
ma = reg.predict(X_test)

In [ ]:
from itertools import count
test_y = pd.DataFrame(zip(np.array(ma), count(0)), columns=['per_square_meter_price', 'id'])
test_y.to_csv('andan-2022-autumn-lab-4/saved_submission_4lab.csv', index=False)
print(test_y)